twitter sentiment analysis

In [5]:
import os
import pandas as pd
import spacy
import random
import torch
import time
from tqdm import tqdm
from spacy.training import Example
from spacy.util import minibatch, compounding
from multiprocessing import Pool

# Vérification du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Charger les données
df_train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
df_test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

# Prétraitement
df_train['Num_words_text'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_train = df_train[df_train['Num_words_text'] >= 3]

# Fonction de préparation des données
def get_training_data(sentiment):
    train_data = []
    for _, row in df_train.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
    return train_data

# Fonction d'entraînement sur GPU avec mesure du temps d'exécution
def train(train_data, output_dir, n_iter=20, model=None):
    start_time = time.time()  # Commencer à mesurer le temps d'exécution
    nlp = spacy.blank("en") if model is None else spacy.load(model)
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")

    # Ajout des labels NER
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # Désactiver les autres pipes pour accélérer l'entraînement
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        nlp.to(device)  # Déplacement du modèle vers le GPU

        # Entraînement avec mise à jour des poids
        for _ in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))  # Taille de mini-lots optimisée
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                examples = [Example.from_dict(nlp.make_doc(text), annotation) for text, annotation in batch]
                nlp.update(examples, drop=0.3, losses=losses)  # Taux de dropout ajusté
            print(f"Losses at iteration {_}: {losses}")

    # Sauvegarde du modèle
    os.makedirs(output_dir, exist_ok=True)
    nlp.to_disk(output_dir)
    print(f"Model saved to {output_dir}")

    # Mesure du temps d'entraînement
    print(f"Training time: {time.time() - start_time:.2f} seconds")

# Entraînement parallèle avec mesure du temps total
def parallel_training():
    start_time = time.time()  # Commencer à mesurer le temps total de l'entraînement
    sentiments = ['positive', 'negative']
    
    pool = Pool(processes=len(sentiments))  # Parallélisme pour chaque sentiment
    results = []
    
    for sentiment in sentiments:
        train_data = get_training_data(sentiment)
        output_dir = f'model_{sentiment}'
        # Appliquer la fonction d'entraînement en parallèle
        results.append(pool.apply_async(train, (train_data, output_dir, 20, None)))
    
    pool.close()  # Fermeture du pool de processus
    pool.join()  # Attendre la fin de l'entraînement parallèle

    # Temps total d'exécution de l'entraînement parallèle
    print(f"Parallel training complete in {time.time() - start_time:.2f} seconds!")

if __name__ == "__main__":
    parallel_training()


Using device: cuda


[2025-01-05 15:04:36,959] [INFO] Created vocabulary
[2025-01-05 15:04:36,962] [INFO] Finished initializing nlp object
[2025-01-05 15:04:38,186] [INFO] Created vocabulary
[2025-01-05 15:04:38,189] [INFO] Finished initializing nlp object


Parallel training complete in 3.32 seconds!
